In [5]:
import json
import os

sam_path = "C:\\Users\\bmirletz\\source\\repos\\sam_dev\\sam\\"
sam_old_path = "C:\\Users\\bmirletz\\source\\repos\\SAM_old\\"

api_path = "api\\api_autogen\\library\\defaults\\"

In [8]:
new_path = sam_path + api_path
file_list_new = []

print(new_path)

for root, dirs, files in os.walk(new_path):
    for file in files:
        file_list_new.append(root + os.sep + file)


C:\Users\bmirletz\source\repos\sam_dev\sam\api\api_autogen\library\defaults\


In [10]:
old_path = sam_old_path + api_path
file_list_old = []

print(old_path)

for root, dirs, files in os.walk(old_path):
    for file in files:
        file_list_old.append(root + os.sep + file)
        

C:\Users\bmirletz\source\repos\SAM_old\api\api_autogen\library\defaults\


In [29]:
def get_flat_dict(defaults_json):
    output = {}
    for k, v in defaults_json.items():
        if type(v) is dict:
            output.update(get_flat_dict(v))
        else:
            output[k] = v
    return output

def file_to_flat_dict(filename):
    with open(filename) as f:
        defaults_json = json.load(f)
        return get_flat_dict(defaults_json)

In [37]:
new_defaults_data = {}
for f in file_list_new:
    file_name = os.path.split(f)[1]
    new_defaults_data[file_name] = file_to_flat_dict(f)
    


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [39]:
old_defaults_data = {}
for f in file_list_old:
    file_name = os.path.split(f)[1]
    old_defaults_data[file_name] = file_to_flat_dict(f)

In [47]:
diffs = []
for k, new_data in new_defaults_data.items():
    if k in old_defaults_data:
        old_data = old_defaults_data[k]
        
        # First, compare sets
        new_key_set = set(new_data.keys())
        old_key_set = set(old_data.keys())
        
        addtl_keys = new_key_set.difference(old_key_set)
        
        if (len(addtl_keys) > 0):
            diffs.append("Configuration " + k + " has new keys " + str(addtl_keys))
        
        removed_keys = old_key_set.difference(new_key_set)
        
        if (len(removed_keys) > 0):
            diffs.append("Configuration " + k + " had the following keys changed or removed " + str(removed_keys))
        
        all_keys = new_key_set.intersection(old_key_set)
        
        for key in all_keys:
            v_new = new_data[key]
            v_old = old_data[key]
            
            if type(v_new) != type(v_old):
                diffs.append("In configuration " + k + " key " + key + " was " + str(type(v_old)) + " but is now " + str(type(v_new)))
            elif v_new != v_old:
                diffs.append("In configuration " + k + " key " + key + " was " + str(v_old) + " but is now " + str(v_new))

        
    else:
        diffs.append("Configuration " + k + " does not exist in old data")
        

In [49]:
with open("version_diffs_2.2.4_to_3.0.0.txt", "w") as out_file:
    for diff in diffs:
        out_file.write(diff + "\n")